# Make HADDOCK Experiments
## Generate run.params, AIR files, and copy resources.

In [1]:
import shutil, os, sys, time
import pandas as pd

In [2]:
candidates = pd.read_excel("../../PD1_Candidates.xlsx")
candidates.head()

,antibody_id,antibody_portion,antibody_sequence_source,antigen,h_chain_version,l_chain_version,h_chain,l_chain,antibody_pdb_path_original,antibody_pdb_path_renumbered,...,haddock_AIRviol,haddock_AIRviol_sd,haddock_dihedviol,haddock_dihedviol_sd,haddock_BSA,haddock_BSA_sd,haddock-score,haddock-score_sd,haddock_prodigy_deltaG_kcalpermol,haddock_prodigy_dissociation_constant_M
0,TUPPD1-001,Fv,"EvoDiff, dd2169c, MSA_OA_DM_MAXSUB",PD1,1,1,EVQLVKSGAEFKKPNDSLKITCKASGYTFTNTGTNVHWVRQAPLKQ...,ELVPTQAIRSLSLFLSEGLKISCSSSRDIDNSSNINTELGSFHTRP...,./docking/inputs/PDBs/diffused_antibodies/comb...,./docking/inputs/PDBs/diffused_antibodies/comb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TUPPD1-002,Fv,"EvoDiff, dd2169c, MSA_OA_DM_MAXSUB",PD1,1,2,EVQLVKSGAEFKKPNDSLKITCKASGYTFTNTGTNVHWVRQAPLKQ...,DIIMTNTPTNLYVSPGESICIICRSSKSGFDGNLVHTYLKWYLQHP...,./docking/inputs/PDBs/diffused_antibodies/comb...,./docking/inputs/PDBs/diffused_antibodies/comb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TUPPD1-003,Fv,"EvoDiff, dd2169c, MSA_OA_DM_MAXSUB",PD1,1,3,EVQLVKSGAEFKKPNDSLKITCKASGYTFTNTGTNVHWVRQAPLKQ...,EIVMTQSPASLPVSPGERATVTCRASESVSDSANGRGWLAWLWQKP...,./docking/inputs/PDBs/diffused_antibodies/comb...,./docking/inputs/PDBs/diffused_antibodies/comb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TUPPD1-004,Fv,"EvoDiff, dd2169c, MSA_OA_DM_MAXSUB",PD1,2,1,QISLVESGPVLVKPNEQLKVACKTSGFTLSSYGTVLNWVRQAPGQR...,ELVPTQAIRSLSLFLSEGLKISCSSSRDIDNSSNINTELGSFHTRP...,./docking/inputs/PDBs/diffused_antibodies/comb...,./docking/inputs/PDBs/diffused_antibodies/comb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TUPPD1-005,Fv,"EvoDiff, dd2169c, MSA_OA_DM_MAXSUB",PD1,2,2,QISLVESGPVLVKPNEQLKVACKTSGFTLSSYGTVLNWVRQAPGQR...,DIIMTNTPTNLYVSPGESICIICRSSKSGFDGNLVHTYLKWYLQHP...,./docking/inputs/PDBs/diffused_antibodies/comb...,./docking/inputs/PDBs/diffused_antibodies/comb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## Define function for creating AIR files
def write_air_file(active1, passive1, active2, passive2, segid1='A', segid2='B', output_file="air.tbl"):
    with open(output_file, "w") as output_file:
        ## Convert residues to integers
        active1 = [int(x) for x in active1]
        passive1 = [int(x) for x in passive1]
        active2 = [int(x) for x in active2]
        passive2 = [int(x) for x in passive2]
        all1 = active1 + passive1
        all2 = active2 + passive2

        ## Write lines from the active1 list
        for resi1 in active1:
            output_file.write('assign (resi {:d} and segid {:s})'.format(resi1, segid1) + '\n')
            output_file.write('(\n')
            c = 0
            for resi2 in all2:
                output_file.write('       (resi {:d} and segid {:s})'.format(resi2, segid2) + '\n')
                c += 1
                if c != len(all2):
                    output_file.write('        or\n')
            output_file.write(') 2.0 2.0 0.0\n\n')

        ## Write lines from the active2 list
        for resi2 in active2:
            output_file.write('assign (resi {:d} and segid {:s})'.format(resi2, segid2) + '\n')
            output_file.write('(\n')
            c = 0
            for resi1 in all1:
                output_file.write('       (resi {:d} and segid {:s})'.format(resi1, segid1) + '\n')
                c += 1
                if c != len(all1):
                    output_file.write('        or\n')
            output_file.write(') 2.0 2.0 0.0\n\n')

    ## File will be closed automatically when exiting the 'with' block


In [4]:
## Define function for Making run.param files
def write_run_params(ambig_tbl = None,
                     haddock_dir = "/root/haddock/haddock2.4-2021-01/",
                     n_comp = 2,
                     pdb_file_1 = "",
                     pdb_file_2 = "",
                     project_dir = "./",
                     prot_segid_1 = "A",
                     prot_segid_2 = "B",
                     run_number = 1,
                     output_file = "run.param"):
    param_lines  = [
        f"HADDOCK_DIR={haddock_dir}",
        f"N_COMP={n_comp}",
        f"PDB_FILE1={pdb_file_1}",
        f"PDB_FILE2={pdb_file_2}",
        f"PROJECT_DIR={project_dir}",
        f"PROT_SEGID_1={prot_segid_1}",
        f"PROT_SEGID_2={prot_segid_2}",
        f"RUN_NUMBER={run_number}",
    ]
    if ambig_tbl:
        param_lines = [f"AMBIG_TBL={ambig_tbl}", *param_lines]
        
    f = open(output_file, "w")
    f.writelines("\n".join(param_lines))
    f.close()

In [5]:
## Make experiment folders
for index, candidate in candidates.iterrows():
    candidate_id = candidate['antibody_id']
    candidate_path = f"../../docking/inputs/experiments/{candidate_id}/"
    os.makedirs(candidate_path, exist_ok=True)

In [6]:
## Write AIR file for each candidate using active and passive residues
for index, candidate in candidates.iterrows():
    candidate_id = candidate['antibody_id']
    target_id = "PD1"
    active1 = candidate['antibody_active_residues'].replace('`','').split(',')
    passive1 = []
    active2 = candidate['antigen_active_residues'].replace('`','').split(',')
    passive2 = []
    candidate_path = f"../../docking/inputs/experiments/{candidate_id}/"
    air_file_name = f"{candidate_id}-{target_id}_air.tbl"
    print(f"Writing AIR file for {candidate_id}...")
    write_air_file(active1, passive1, active2, passive2, segid1='A', segid2='B', output_file = f"{candidate_path}{air_file_name}")


Writing AIR file for TUPPD1-001...
Writing AIR file for TUPPD1-002...
Writing AIR file for TUPPD1-003...
Writing AIR file for TUPPD1-004...
Writing AIR file for TUPPD1-005...
Writing AIR file for TUPPD1-006...
Writing AIR file for TUPPD1-007...
Writing AIR file for TUPPD1-008...
Writing AIR file for TUPPD1-009...
Writing AIR file for Acrixolimab...
Writing AIR file for Balstilimab...
Writing AIR file for Budigalimab...
Writing AIR file for Camrelizumab...
Writing AIR file for Cemiplimab...
Writing AIR file for Cetrelimab...
Writing AIR file for Dostarlimab...
Writing AIR file for Enlonstobart...
Writing AIR file for Eurestobart...
Writing AIR file for Ezabenlimab...
Writing AIR file for Finotonlimab...
Writing AIR file for Geptanolimab...
Writing AIR file for Iparomlimab...
Writing AIR file for Lipustobart...
Writing AIR file for Nivolumab...
Writing AIR file for Nofazinlimab...
Writing AIR file for Pembrolizumab...
Writing AIR file for Penpulimab...
Writing AIR file for Perenostobart.

In [7]:
## Write run.param file for each candidate
for index, candidate in candidates.iterrows():
    candidate_id = candidate['antibody_id']
    target_id = "PD1"
    candidate_path = f"../../docking/inputs/experiments/{candidate_id}/"
    write_run_params(ambig_tbl = f"{candidate_id}-{target_id}_air.tbl",
                     haddock_dir = "/root/haddock/haddock2.4-2021-01/",
                     n_comp = 2,
                     pdb_file_1 = f"./{candidate_id}_renumberedLplus1000.pdb",
                     pdb_file_2 = "./5jxe_pd1.pdb",
                     project_dir = "./",
                     prot_segid_1 = "A",
                     prot_segid_2 = "B",
                     run_number = 1,
                     output_file = f"{candidate_path}run.param")

In [8]:
## Copy HADDOCK Scripts and Target PDB to each candidate directory
for index, candidate in candidates.iterrows():
    candidate_id = candidate['antibody_id']
    candidate_path = f"../../docking/inputs/experiments/{candidate_id}/"
    shutil.copyfile("./haddock_resources/run-docking.csh", f"{candidate_path}run-docking.csh")
    shutil.copytree("./haddock_resources/ana_scripts", f"{candidate_path}ana_scripts", dirs_exist_ok=True)
    shutil.copyfile("../../docking/inputs/PDBs/antigens/PD1/5jxe_pd1.pdb", f"{candidate_path}5jxe_pd1.pdb")
    shutil.copyfile("./haddock_resources/cleanup.py", f"{candidate_path}cleanup.py")
    if "TUPPD1" in candidate_id:
        shutil.copyfile(f"../../docking/inputs/PDBs/diffused_antibodies/combined/{candidate_id}/{candidate_id}_renumberedLplus1000.pdb", f"{candidate_path}{candidate_id}_renumberedLplus1000.pdb")
    else:
        shutil.copyfile(f"../../docking/inputs/PDBs/reference_antibodies/{candidate_id}_renumberedLplus1000.pdb", f"{candidate_path}{candidate_id}_renumberedLplus1000.pdb")